In [1]:
import pandas as pd
import numpy as np

In [2]:
real_sales =  pd.DataFrame.from_csv('sales-jan-mar-2017.csv').reset_index()
real_sales['Salesdate'] = pd.to_datetime(real_sales['Salesdate'], format='%Y-%m-%d %H:%M:%S')

In [3]:
real_sales_mar = real_sales[real_sales['Month']==3]

In [4]:
sales_estimated_ml =  pd.DataFrame.from_csv('sales_march_ml.csv').reset_index()
sales_estimated_ml['Salesdate'] = sales_estimated_ml['Salesdate'].str.replace('/', '-')
sales_estimated_ml['Salesdate'] = pd.to_datetime(sales_estimated_ml['Salesdate'], format='%d-%m-%Y')

In [5]:
sales_estimated_avg =  pd.DataFrame.from_csv('sales_march_moving_average.csv').reset_index()
sales_estimated_avg['Salesdate'] = sales_estimated_avg['Salesdate'].str.replace('/', '-')
sales_estimated_avg['Salesdate'] = pd.to_datetime(sales_estimated_avg['Salesdate'], format='%d-%m-%Y')

In [6]:
sales_with_ml = pd.merge(real_sales_mar, sales_estimated_ml, on=['Locationid','RecipeName','PLU','Year','Month','Day','WeekDay','YearDay'], how='inner')
sales_with_ml_avg = pd.merge(sales_with_ml, sales_estimated_avg, on=['Locationid','RecipeName','PLU','Year','Month','Day','WeekDay','YearDay'], how='inner')
sales_with_ml_avg = sales_with_ml_avg.rename(columns={'Quantity_x': 'real', 'Quantity_y':'ml', 'Quantity':'avg'})

In [7]:
products = sales_with_ml_avg['PLU'].unique().tolist()
locations = sales_with_ml_avg['Locationid'].unique().tolist()

In [8]:
correl = sales_with_ml_avg[['real','ml','avg']].corr()
all_population = correl*correl
all_population

,real,ml,avg
real,1.000000,0.230925,0.341160
ml,0.230925,1.000000,0.237554
avg,0.341160,0.237554,1.000000


In [9]:
corr_by_groups = sales_with_ml_avg.groupby(['Locationid','RecipeName','PLU'])['real', 'ml', 'avg'].corr()
corr_by_groups = corr_by_groups*corr_by_groups

In [11]:
d = corr_by_groups.reset_index()
d = d[d['level_3']=='real']
rows_all = len(d.index)
print("Number of all observations = %d" % (rows_all))

ml_better = d[d['ml'] > d['avg']]
ml_diff = ml_better['ml'] - ml_better['avg']
rows_ml = len(ml_better.index)
print("ML is better in %d observations of %d (%d%%)" % (rows_ml, rows_all, rows_ml/rows_all*100))
print("ML performance is better in %d%% for above observations" % (ml_diff.mean()*100))

avg_better = d[d['ml'] < d['avg']]
avg_diff = avg_better['avg'] - avg_better['ml']
rows_avg = len(avg_better.index)
print("AVG is better in %d observations of %d (%d%%)" % (rows_avg, rows_all, rows_avg/rows_all*100))
print("AVG performance is better in %d%% for above observations" % (avg_diff.mean()*100))

Number of all observations = 2325
ML is better in 1526 observations of 2325 (65%)
ML perfomance is better in 11% for above observations
AVG is better in 799 observations of 2325 (34%)
AVG perfomance is better in 5% for above observations
